In [101]:
import torch
import os 
from torch.utils.mobile_optimizer import optimize_for_mobile
import numpy as np
import matplotlib.pyplot as plt

- load in weather data using only valid columns, leaving out the last column (wind chill label)
- create a tensor from all data

In [104]:
weather_data = np.genfromtxt("2022_data.csv",delimiter = ",", usecols = (0,1,2,3,4,6,8,9,11,12,15,17,18,19,22), skip_header = 1, filling_values=0.0)[:, :-1]
weather_tensor = torch.tensor(weather_data).float()
xdims = weather_tensor.shape[1] - 1


Make a pytorch DataSet from the tensor

In [105]:
weather_dataset = torch.utils.data.TensorDataset(weather_tensor[:, :-1], weather_tensor[:, -1:])


DataLoader 
puts data on the right device, shuffles, can use parallel programming, define batch size etc
- sees everything in an epoch
- next epoch, sees them in a different order
- suffles with 32 irows in a batch


In [106]:
loader = torch.utils.data.DataLoader(weather_dataset, shuffle=True, batch_size=32)


In [107]:
net = torch.nn.Sequential(
    torch.nn.Linear(xdims, 32), # takes in x dims columns, then funnels to a hidden layer of 32
    torch.nn.ELU(), 
    # ELU is an activation layer, like a sigmoidal function!
    torch.nn.Linear(32, 32),
    torch.nn.ELU(),
    torch.nn.Linear(32, 16),
    torch.nn.ELU(),
    torch.nn.Linear(16, 1)
)


In [109]:
# Mean squared error loss function
criterion = torch.nn.MSELoss()


In [110]:
# optimizer called Adam, using a learning rate of 1e-4
opt = torch.optim.Adam(net.parameters(), 1e-4 )


Training loop

In [130]:
for ep in range(1000):
    # every epoch sees every row exactly one time, but inputs are shuffled every epoch
    total_loss = 0.0
    
    # batch by batch
    for batch in loader:
        
        opt.zero_grad()
        
        batchX = batch[0]
        batchY = batch[1]

        pred = net(batchX)
        loss = criterion(pred, batchY)
        
        loss.backward()
        opt.step()
        total_loss += loss



In [131]:
## This code will print out the real, then expected for every batch of 32 
# for batch in loader:
#     batchX = batch[0]
#     batchY = batch[1]
#     print(batchY)
#     print(net(batchX))
    

In [120]:
# optimize for mobile so that we can export and use in Swift app
example_tensor = weather_tensor[0,:-1]
traced_script_module = torch.jit.trace(net, example_tensor)
torchscript_model_optimized = optimize_for_mobile(traced_script_module)

# save as .pt 
path = os.path.join(os.getcwd(),"model.pt")
torchscript_model_optimized._save_for_lite_interpreter(path)



### Test

In [132]:
test = weather_tensor[0,:-1]
label = weather_tensor[0,-1]
label

tensor(4.9000)

In [133]:
net(test)


tensor([3.8136], grad_fn=<AddBackward0>)

NOTES FROM CORY: HOW TO SPLIT UP training, validation, testing

def train(loader):
    one tound of training with all data in the loader
    
    
def eval(loader):
    feed evyerhting in batches to model, adds up and takes the losses of the batches
    

